In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv, find_dotenv
import os
from datetime import date, timedelta
import plotly.express as px

pd.set_option("display.max_columns", None)


In [3]:
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
FACEIT_PLAYERID = os.environ.get("FACEIT_PLAYERID")


In [4]:
DF_all = pd.read_pickle("./AllDatabz2.pkl", compression="bz2")


In [5]:
Map_names = ['de_cache', 'de_cbble', 'de_dust2', 'de_inferno', 'de_mirage','de_nuke', 'de_overpass', 'de_train']

In [6]:
DF_all = DF_all[DF_all['Map'].isin(Map_names)]
DF_all["K/D"] = DF_all["Kills"] / DF_all["Deaths"]
DF_all["K/A"] = DF_all["Kills"] / DF_all["Assists"]
DF_all = DF_all.query("Rounds > 16")
DF_all = DF_all[~DF_all["K/D"].isin([np.inf, -np.inf, np.nan])]
DF_all = DF_all[~DF_all["K/A"].isin([np.inf, -np.inf, np.nan])]



In [21]:
def count_wins(x):
    wins = x.value_counts().get("Win")
    if wins:
        return wins
    else:
        return 0

In [22]:
q_low = DF_all["Kills"].quantile(0.01)
q_hi = DF_all["Kills"].quantile(0.99)

DF_all = DF_all[(DF_all["Kills"] < q_hi) & (DF_all["Kills"] > q_low)]

In [23]:
DF_me = DF_all.query("PlayerId == @FACEIT_PLAYERID")
DF_others:pd.DataFrame = DF_all.query("PlayerId != @FACEIT_PLAYERID")

In [25]:
Maps = (
    DF_others.groupby(by=["Map"])
    .aggregate(
        {
            "Kills": "sum",
            "Deaths": "sum",
            "Mvps": "sum",
            "Map": "count",
            "Headshots%": "mean",
            "Result": lambda x: count_wins(x), 
        }
    )
    .rename({"Map": "TimesPlayed"}, axis=1)
)


In [ ]:
fig = px.bar(Maps, y="TimesPlayed", x=Maps.index,
            text_auto = ".2s",

             
            title="Most played maps for other players from Faceit"
            )
fig.update_layout(xaxis_tickangle=-30)

fig.show()